In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

API_KEY = ''  # Replace with your actual key

# Get SPY components via FMP ETF holdings endpoint
def get_spy_holdings():
    url = f"https://financialmodelingprep.com/api/v3/etf-holder/SPY?apikey={API_KEY}"
    response = requests.get(url)
    response.raise_for_status()
    holdings = response.json()
    tickers = [item['asset'] for item in holdings if 'asset' in item]
    return list(set(tickers))

# Fetch historical prices for all tickers at once
def fetch_bulk_price_data(tickers, start_date, end_date):
    all_data = {}

    chunk_size = 50  # FMP API limit
    chunks = [tickers[i:i + chunk_size] for i in range(0, len(tickers), chunk_size)]

    for chunk in chunks:
        symbols = ",".join(chunk)
        url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{symbols}?from={start_date}&to={end_date}&apikey={API_KEY}&serietype=line"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        # If only one ticker is returned, wrap it in a list
        if isinstance(data, dict) and 'symbol' in data:
            data = [data]
        elif isinstance(data, dict) and 'historicalStockList' in data:
            data = data['historicalStockList']

        for stock_data in data:
            ticker = stock_data.get("symbol")
            hist = stock_data.get("historical")
            if not hist or not ticker:
                continue
            df = pd.DataFrame(hist)
            df['date'] = pd.to_datetime(df['date'])
            df.set_index('date', inplace=True)
            df.sort_index(inplace=True)
            all_data[ticker] = df['close']

    return pd.DataFrame(all_data).dropna(axis=1)



In [12]:
# Define backtest window
end_date = datetime.now()
start_date = end_date - timedelta(days=5 * 365)  # Past 5 years
# Backtest period
end_date = datetime.now()
start_date = end_date - timedelta(days=5*365)

# Get SPY tickers and price data
spy_tickers = get_spy_holdings()
price_data = fetch_bulk_price_data(spy_tickers, start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))

# Clean index
price_data.index = price_data.index.tz_localize(None)

# Get tickers and download data
excluded_tickers = ['BF-B', 'BRK-B', 'ZBRA']  # Adjusted to FMP-style dashes
tickers = get_sp500_tickers(excluded_tickers)
price_data = build_price_dataset(tickers, start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'), limit=500)

# Ensure time index is timezone-naive
price_data.index = price_data.index.tz_localize(None)


Fetching 1/500: MMM
Fetching 2/500: AOS
Fetching 3/500: ABT
Fetching 4/500: ABBV
Fetching 5/500: ACN
Fetching 6/500: ADBE
Fetching 7/500: AMD
Fetching 8/500: AES
Fetching 9/500: AFL
Fetching 10/500: A
Fetching 11/500: APD
Fetching 12/500: ABNB
Fetching 13/500: AKAM
Fetching 14/500: ALB
Fetching 15/500: ARE
Fetching 16/500: ALGN
Fetching 17/500: ALLE
Fetching 18/500: LNT
Fetching 19/500: ALL
Fetching 20/500: GOOGL
Fetching 21/500: GOOG
Fetching 22/500: MO
Fetching 23/500: AMZN
Fetching 24/500: AMCR
Fetching 25/500: AEE
Fetching 26/500: AEP
Fetching 27/500: AXP
Fetching 28/500: AIG
Fetching 29/500: AMT
Fetching 30/500: AWK
Fetching 31/500: AMP
Fetching 32/500: AME
Fetching 33/500: AMGN
Fetching 34/500: APH
Fetching 35/500: ADI
Fetching 36/500: ANSS
Fetching 37/500: AON
Fetching 38/500: APA
Fetching 39/500: APO
Fetching 40/500: AAPL
Fetching 41/500: AMAT
Fetching 42/500: APTV
Fetching 43/500: ACGL
Fetching 44/500: ADM
Fetching 45/500: ANET
Fetching 46/500: AJG
Fetching 47/500: AIZ
Fetchin

In [15]:
def backtest_risk_adj_momentum(price_data, top_n=3, months=60, momentum_days=60, vol_days=30):
    monthly_returns = []
    monthly_dates = []

    for i in range(months):
        current_start = price_data.index[0] + timedelta(days=i*30)
        current_end = current_start + timedelta(days=30)

        lookback = price_data.loc[:current_start]
        if lookback.shape[0] < momentum_days + 1:
            continue

        momentum = lookback.pct_change(momentum_days).iloc[-1]
        volatility = lookback.pct_change().rolling(vol_days).std().iloc[-1]
        scores = momentum / volatility

        top_tickers = scores.dropna().nlargest(top_n).index
        forward = price_data.loc[current_start:current_end]

        if forward.empty or not all(t in forward.columns for t in top_tickers):
            continue

        forward_returns = forward[top_tickers].pct_change().mean(axis=1)
        monthly_return = (1 + forward_returns).prod() - 1

        monthly_dates.append(current_end)
        monthly_returns.append(monthly_return)

    cumulative_return = (1 + pd.Series(monthly_returns)).prod() - 1

    print("Monthly Performance:")
    for d, r in zip(monthly_dates, monthly_returns):
        print(f"{d.strftime('%Y-%m-%d')}: {r:.2%}")

    print(f"\nCumulative Return (5 Years): {cumulative_return:.2%}")
    return monthly_dates, monthly_returns, cumulative_return


In [16]:
# Run full backtest
dates, rets, total_ret = backtest_risk_adj_momentum(price_data)

# Calculate current year performance
def calc_ytd(dates, rets):
    year = datetime.now().year
    ytd = [r for d, r in zip(dates, rets) if d.year == year]
    ytd_return = (1 + pd.Series(ytd)).prod() - 1
    print(f"\nYTD Return ({year}): {ytd_return:.2%}")
    return ytd_return

# Get YTD performance
calc_ytd(dates, rets)


Monthly Performance:
2020-10-28: -9.60%
2020-11-27: 20.29%
2020-12-27: 4.24%
2021-01-26: 7.89%
2021-02-25: 3.01%
2021-03-27: 7.48%
2021-04-26: 2.86%
2021-05-26: -1.57%
2021-06-25: -0.89%
2021-07-25: 5.78%
2021-08-24: 4.95%
2021-09-23: 0.61%
2021-10-23: -3.34%
2021-11-22: 5.26%
2021-12-22: 4.76%
2022-01-21: -12.20%
2022-02-20: 1.98%
2022-03-22: -0.35%
2022-04-21: 6.58%
2022-05-21: -18.26%
2022-06-20: -14.18%
2022-07-20: 6.65%
2022-08-19: 8.00%
2022-09-18: -3.65%
2022-10-18: -13.72%
2022-11-17: 18.79%
2022-12-17: -7.27%
2023-01-16: 5.43%
2023-02-15: 3.03%
2023-03-17: -5.97%
2023-04-16: 8.63%
2023-05-16: -0.93%
2023-06-15: 0.94%
2023-07-15: -2.44%
2023-08-14: -7.63%
2023-09-13: 3.93%
2023-10-13: 1.15%
2023-11-12: -5.00%
2023-12-12: 7.66%
2024-01-11: 1.73%
2024-02-10: 0.43%
2024-03-11: 0.70%
2024-04-10: -5.54%
2024-05-10: -0.61%
2024-06-09: -5.17%
2024-07-09: -3.16%
2024-08-08: -0.62%
2024-09-07: 2.70%
2024-10-07: 0.44%
2024-11-06: -3.56%
2024-12-06: 6.69%
2025-01-05: -6.16%
2025-02-04: 7.

np.float64(-0.07495895641653905)